# Quant Bio Project #2

## Setup

In [5]:
import pandas as pd
import numpy as np
import csv

## Reading in the Data

In [15]:
bin_mat = pd.read_csv('BinMat.csv', header=None)
eval_mat = pd.read_csv('EvalMat.csv', header=None)

In [16]:
# Matlab split each character in the gene names into a separate column,
# so rather than try to deal with Matlab I am just going to read the
# file as is and then put the gene name back together
with open('gene_names.csv') as f:
    reader = csv.reader(f)
    raw_gene_names = list(reader)

gene_names = []
for gene in raw_gene_names:
    gene_names.append(''.join(gene))

In [17]:
# Same thing for the species names
with open('species_rows.csv') as f:
    reader = csv.reader(f)
    raw_species_rows = list(reader)
    
species_rows = []
for species in raw_species_rows:
    species_rows.append(''.join(species))

In [20]:
# Now we put the gene and species info with our
# gene presence/absence data
bin_mat.columns = gene_names
eval_mat.columns = gene_names